# Tests extracción de rasgos estilométricos con RAID

In [1]:
# import numpy as np
# import math
# import random
# import matplotlib.pyplot as plt
# import cvxopt
# import seaborn as sns
# import soundfile as sf
# import itertools
# import pywt
# import librosa
# import librosa.display
# from IPython.display import Audio
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score
# from cvxopt import matrix, solvers
# import pandas as pd
# import os
# import scipy
# import scipy.signal
# from sklearn.decomposition import PCA, TruncatedSVD, FactorAnalysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.model_selection import train_test_split
# from sklearn.random_projection import SparseRandomProjection, johnson_lindenstrauss_min_dim, GaussianRandomProjection
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# import torch
# from torch import nn
# import torch.optim as optim
# import torch.nn.functional as F
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

In [17]:
import stylo_metrix as sm
import pandas as pd
from raid import run_detection, run_evaluation
from raid.utils import load_data

import shutil
import os
from processer import split_text_into_sentences

## Manejo de RAID

In [18]:
# Download the RAID dataset without adversarial attacks
or_train_noadv_df = load_data(split="train", include_adversarial=False)
# test_noadv_df = load_data(split="test", include_adversarial=False)
# extra_noadv_df = load_data(split="extra", include_adversarial=False)

In [4]:
display(or_train_noadv_df.head(7))

id                         adv_source_id  \
0  e5e058ce-be2b-459d-af36-32532aaba5ff  e5e058ce-be2b-459d-af36-32532aaba5ff   
1  f95b107b-d176-4af5-90f7-4d0bb20caf93  f95b107b-d176-4af5-90f7-4d0bb20caf93   
2  856d8972-9e3d-4544-babc-0fe16f21e04d  856d8972-9e3d-4544-babc-0fe16f21e04d   
3  fbc8a5ea-90fa-47b8-8fa7-73dd954f1524  fbc8a5ea-90fa-47b8-8fa7-73dd954f1524   
4  72c41b8d-0069-4886-b734-a4000ffca286  72c41b8d-0069-4886-b734-a4000ffca286   
5  72fe360b-cce6-4daf-b66a-1d778f5964f8  72fe360b-cce6-4daf-b66a-1d778f5964f8   
6  df594cf4-9a0c-4488-bcb3-68f41e2d5a16  df594cf4-9a0c-4488-bcb3-68f41e2d5a16   

                              source_id  model decoding repetition_penalty  \
0  e5e058ce-be2b-459d-af36-32532aaba5ff  human      NaN                NaN   
1  f95b107b-d176-4af5-90f7-4d0bb20caf93  human      NaN                NaN   
2  856d8972-9e3d-4544-babc-0fe16f21e04d  human      NaN                NaN   
3  fbc8a5ea-90fa-47b8-8fa7-73dd954f1524  human      NaN                NaN   
4  72c41b8d-0069-4886-b734-a4000ffca286  human      NaN                NaN   
5  72fe360b-cce6-4daf-b66a-1d778f5964f8  human      NaN                NaN   
6  df594cf4-9a0c-4488-bcb3-68f41e2d5a16  human      NaN                NaN   

  attack     domain                                              title prompt  \
0   none  abstracts  FUTURE-AI: Guiding Principles and Consensus Re...    NaN   
1   none  abstracts  EdgeFlow: Achieving Practical Interactive Segm...    NaN   
2   none  abstracts  Semi-supervised Contrastive Learning for Label...    NaN   
3   none  abstracts  Combo Loss: Handling Input and Output Imbalanc...    NaN   
4   none  abstracts  Attention-Based 3D Seismic Fault Segmentation ...    NaN   
5   none  abstracts   Segmenter: Transformer for Semantic Segmentation    NaN   
6   none  abstracts  Mining Contextual Information Beyond Image for...    NaN   

                                          generation  
0  The recent advancements in artificial intellig...  
1  High-quality training data play a key role in ...  
2  The success of deep learning methods in medica...  
3  Simultaneous segmentation of multiple organs f...  
4  Detection faults in seismic data is a crucial ...  
5  Image segmentation is often ambiguous at the l...  
6  This paper studies the context aggregation pro...

In [19]:
intCols = ['id','model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
# train_noadv_df = or_train_noadv_df[or_train_noadv_df['model'] != 'human']

# Copia del dataframe con columnas específicas
train_noadv_df = or_train_noadv_df.copy()
train_noadv_df = train_noadv_df[intCols]

display(train_noadv_df.head(7))

id  model     domain  \
0  e5e058ce-be2b-459d-af36-32532aaba5ff  human  abstracts   
1  f95b107b-d176-4af5-90f7-4d0bb20caf93  human  abstracts   
2  856d8972-9e3d-4544-babc-0fe16f21e04d  human  abstracts   
3  fbc8a5ea-90fa-47b8-8fa7-73dd954f1524  human  abstracts   
4  72c41b8d-0069-4886-b734-a4000ffca286  human  abstracts   
5  72fe360b-cce6-4daf-b66a-1d778f5964f8  human  abstracts   
6  df594cf4-9a0c-4488-bcb3-68f41e2d5a16  human  abstracts   

                                               title prompt  \
0  FUTURE-AI: Guiding Principles and Consensus Re...    NaN   
1  EdgeFlow: Achieving Practical Interactive Segm...    NaN   
2  Semi-supervised Contrastive Learning for Label...    NaN   
3  Combo Loss: Handling Input and Output Imbalanc...    NaN   
4  Attention-Based 3D Seismic Fault Segmentation ...    NaN   
5   Segmenter: Transformer for Semantic Segmentation    NaN   
6  Mining Contextual Information Beyond Image for...    NaN   

                                          generation  
0  The recent advancements in artificial intellig...  
1  High-quality training data play a key role in ...  
2  The success of deep learning methods in medica...  
3  Simultaneous segmentation of multiple organs f...  
4  Detection faults in seismic data is a crucial ...  
5  Image segmentation is often ambiguous at the l...  
6  This paper studies the context aggregation pro...

In [6]:
print("Información del dataset de entrenamiento:")
print(f"Forma del dataset: {train_noadv_df.shape}")
print(f"Columnas: {list(train_noadv_df.columns)}")
print(f"Modelos unicos: {train_noadv_df['model'].unique()}")
print(f"Dominios unicos: {train_noadv_df['domain'].unique()}")

Información del dataset de entrenamiento:
Forma del dataset: (467985, 6)
Columnas: ['id', 'model', 'domain', 'title', 'prompt', 'generation']
Modelos unicos: ['human' 'llama-chat' 'mpt' 'mpt-chat' 'gpt2' 'mistral' 'mistral-chat'
 'gpt3' 'cohere' 'chatgpt' 'gpt4' 'cohere-chat']
Dominios unicos: ['abstracts' 'books' 'news' 'poetry' 'recipes' 'reddit' 'reviews' 'wiki']


In [7]:
# intCols = ['model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
filtered = train_noadv_df[train_noadv_df['model'] != 'human']

display(filtered.head(7))

id       model     domain  \
493  2bd98bd7-3356-43bf-8c5d-69ef336d0536  llama-chat  abstracts   
494  e8bdc461-3ff2-4d68-8c7b-cdbc086f62b3  llama-chat  abstracts   
495  ee968d29-ce73-4c5d-804d-0a0efec4bea4  llama-chat  abstracts   
496  3d24eb90-f540-490f-81c8-e4a24fd49ad7  llama-chat  abstracts   
497  5f817bbe-4fb4-4011-a1e9-fcf12990f450  llama-chat  abstracts   
498  7389e65b-2e27-4b90-999a-53e28b773315  llama-chat  abstracts   
499  8b79a378-67db-48e8-8950-4d3215cfef16  llama-chat  abstracts   

                                                 title  \
493  FUTURE-AI: Guiding Principles and Consensus Re...   
494  FUTURE-AI: Guiding Principles and Consensus Re...   
495  EdgeFlow: Achieving Practical Interactive Segm...   
496  EdgeFlow: Achieving Practical Interactive Segm...   
497  Semi-supervised Contrastive Learning for Label...   
498  Semi-supervised Contrastive Learning for Label...   
499  Combo Loss: Handling Input and Output Imbalanc...   

                                                prompt  \
493  Write the abstract for the academic paper titl...   
494  Write the abstract for the academic paper titl...   
495  Write the abstract for the academic paper titl...   
496  Write the abstract for the academic paper titl...   
497  Write the abstract for the academic paper titl...   
498  Write the abstract for the academic paper titl...   
499  Write the abstract for the academic paper titl...   

                                            generation  
493  In the paper "FUTURE-AI: Guiding Principles an...  
494  In the paper "Future-AI: Guiding Principles an...  
495  In this paper, we present EdgeFlow, a novel ap...  
496  In this paper, we present a novel approach to ...  
497  In this paper, we propose a novel approach to ...  
498  In this paper, we propose a novel approach to ...  
499  In the field of medical image segmentation, im...

In [8]:
# Opción 1: Configurar pandas para mostrar más contenido
pd.set_option('display.max_colwidth', None)  # Sin límite de ancho
pd.set_option('display.max_rows', None)      # Sin límite de filas (usar con cuidado)

complex_filter = train_noadv_df[(train_noadv_df['model'] == 'human') &
                                (train_noadv_df['domain'] == 'abstracts')]

# Ahora al mostrar el dataframe, verás el contenido completo
display(complex_filter[['model', 'domain', 'generation']].head(3))

# Si quieres restaurar los valores por defecto de pandas:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

model     domain  \
0  human  abstracts   
1  human  abstracts   
2  human  abstracts   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        generation  
0  The recent advancements in artificial intelligence (AI) combined with the\nextensive amount of data generated by today's clinical systems, has led to the\ndevelopment of imaging AI solutions across the whole value chain of medical\nimaging, including image reconstruction, medical image segmentation,\nimage-based diagnosis and treatment planning. Notwithstanding the successes and\nfuture potential of AI in medical imaging, many stakeholders are concerned of\nthe potential risks and ethical implications of imaging AI solutions, which are\nperceived as complex, opaque, and difficult to comprehend, utilise, and trust\nin critical clinical applications. Despite these concerns and risks, there are\ncurrently no concrete guidelines and best practices for guiding future AI\ndevelopments in medical imaging towards increased trust, safety and adoption.\nTo bridge this gap, this paper introduces a careful selection of guiding\nprinciples drawn from the accumulated experiences, consensus, and best\npractices from five large European projects on AI in Health Imaging. These\nguiding principles are named FUTURE-AI and its building blocks consist of (i)\nFairness, (ii) Universality, (iii) Traceability, (iv) Usability, (v) Robustness\nand (vi) Explainability. In a step-by-step approach, these guidelines are\nfurther translated into a framework of concrete recommendations for specifying,\ndeveloping, evaluating, and deploying technically, clinically and ethically\ntrustworthy AI solutions into clinical practice.  
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       High-quality training data play a key role in image segmentation tasks.\nUsually, pixel-level annotations are expensive, laborious and time-consuming\nfor the large volume of training data. To reduce labelling cost and improve\nsegmentation quality, interactive segmentation methods have been proposed,\nwhich provide the result with just a few clicks. However, their perf

In [9]:
# Opción 5: Filtrar con condiciones complejas (OR entre grupos)
complex_filter = train_noadv_df[(train_noadv_df['model'] == 'human') | 
                                ((train_noadv_df['model'] == 'gpt4') & 
                                 (train_noadv_df['domain'] == 'books'))]
print(f"\nFiltro complejo: {len(complex_filter)}")
print(complex_filter['model'].value_counts())


Filtro complejo: 16933
model
human    13371
gpt4      3562
Name: count, dtype: int64


## Uso de Stylometrix

### Test de extracción

In [20]:
# Get sample dataframe 'generation'

filtered_by_domain = train_noadv_df[
    (train_noadv_df['domain'] != 'recipes')
    ]
generation_sample = filtered_by_domain[['id', 'model', 'domain', 'generation']].sample(n=5, random_state=3)

display(generation_sample)

id       model     domain  \
110956  c4059838-c14e-4b84-b75c-12bc0bd2f34a      cohere      books   
363000  c5001dc3-5baa-4abf-8a1b-2e0bf6d7c77d        gpt3     reddit   
172379  42bb8f1e-60ec-428d-800c-e9059a0efdab        gpt3       news   
200619  6a2721a4-d15e-44a9-9f09-e48e60e17a4d  llama-chat     poetry   
27573   4a64da9c-2df8-4f58-91ee-2f9192cc5667     mistral  abstracts   

                                               generation  
110956   The story centers on Charles, the husband of ...  
363000  \n\nJust wondering where do I start?\n\nI've b...  
172379  \n\nLasers are being used to create an ultra-f...  
200619  With eager hands, we plant the seeds,\nIn fert...  
27573   Medical image analysis has experienced an expl...

In [11]:
print(generation_sample['generation'].iloc[0])
outputList = split_text_into_sentences(generation_sample['generation'].iloc[0])

 The story centers on Charles, the husband of Alice, who is visiting his new estate. He insists that she stay in the chateau while he travels to Paris on business. Charles abruptly ends his trip and returns home, surprising Alice. He announces that he has sold the estate, and they will have to leave at once. Then he hands Alice a key he says he found on the path leading to the chateau.

Alice rejoices at the news of their departure, but is perplexed by the key. Charles vehemently tells her to discard it, but she cannot bring herself to throw it away. Later, while exploring the chateau, she discovers a room that is locked. Curiosity gets the better of her, and she tries the key. It fits! Alice is shocked to discover the room is full of blood-stained weapons and dead bodies. In a moment of madness, she realizes that Charles has killed his previous wives, and she must dispose of the key lest she become his next victim.

Alice returns the key to Charles and tells him it does not fit any of

#### Extracción de rasgos en un registro

In [12]:
for sentence in outputList:
    print(sentence)
    print("-----")

The story centers on Charles, the husband of Alice, who is visiting his new estate.
-----
He insists that she stay in the chateau while he travels to Paris on business.
-----
Charles abruptly ends his trip and returns home, surprising Alice.
-----
He announces that he has sold the estate, and they will have to leave at once.
-----
Then he hands Alice a key he says he found on the path leading to the chateau.
-----
Alice rejoices at the news of their departure, but is perplexed by the key.
-----
Charles vehemently tells her to discard it, but she cannot bring herself to throw it away.
-----
Later, while exploring the chateau, she discovers a room that is locked.
-----
Curiosity gets the better of her, and she tries the key.
-----
It fits!
-----
Alice is shocked to discover the room is full of blood-stained weapons and dead bodies.
-----
In a moment of madness, she realizes that Charles has killed his previous wives, and she must dispose of the key lest she become his next victim.
-----


In [13]:
# Borrar contenido de la carpeta de salida antes de guardar nuevos resultados
def reset_output_dir(output_dir):
    # output_dir = "testVisual/single"
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)


In [14]:
output_dir = "testVisual/single"
reset_output_dir(output_dir)

stylo = sm.StyloMetrix('en', debug=True, save_path=output_dir) # define langauge, one of ('de','en', 'pl', 'ru', 'ukr')
metrics = stylo.transform(outputList) # list of texts
metrics

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
  0%|          | 0/23 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 23/23 [00:03<00:00,  6.81it/s]


File saved in location: testVisual\single\sm_output1.csv
File saved in location: testVisual\single\sm_debug1.csv


(                                                 text  POS_VERB  POS_NOUN  \
 0   The story centers on Charles, the husband of A...  0.200000  0.200000   
 1   He insists that she stay in the chateau while ...  0.200000  0.133333   
 2   Charles abruptly ends his trip and returns hom...  0.300000  0.100000   
 3   He announces that he has sold the estate, and ...  0.375000  0.062500   
 4   Then he hands Alice a key he says he found on ...  0.235294  0.176471   
 5   Alice rejoices at the news of their departure,...  0.214286  0.214286   
 6   Charles vehemently tells her to discard it, bu...  0.312500  0.000000   
 7   Later, while exploring the chateau, she discov...  0.333333  0.166667   
 8   Curiosity gets the better of her, and she trie...  0.181818  0.181818   
 9                                            It fits!  0.500000  0.000000   
 10  Alice is shocked to discover the room is full ...  0.333333  0.266667   
 11  In a moment of madness, she realizes that Char...  0.222222

#### Extracción de rasgos en múltiples registros

In [ ]:
output_dir = "testVisual/multiple"
reset_output_dir(output_dir)

stylo = sm.StyloMetrix('en', debug=True, save_path=output_dir)

for idx, i in enumerate(generation_sample.index):
    print(f"Processing index: {i} (iteration {idx})")
    text = generation_sample['generation'].loc[i]
    sentences = split_text_into_sentences(text)
    
    # Cambiar los nombres de los archivos para cada iteración
    stylo.output_name = f"output_iter{idx}_idx{i}"
    stylo.debug_name = f"debug_iter{idx}_idx{i}"
    features = stylo.transform(sentences)
    print(f"Saved: {stylo.output_name}.csv and {stylo.debug_name}.csv")    

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Processing index: 110956 (iteration 0)
[OK] Total de oraciones: 23


  0%|          | 0/23 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 23/23 [00:03<00:00,  7.02it/s]


File saved in location: testVisual\multiple\output_iter0_idx1109561.csv
File saved in location: testVisual\multiple\debug_iter0_idx1109561.csv
Saved: output_iter0_idx110956.csv and debug_iter0_idx110956.csv
Processing index: 363000 (iteration 1)
[OK] Total de oraciones: 5


100%|██████████| 5/5 [00:00<00:00, 11.08it/s]


...

File saved in location: testVisual\multiple\output_iter2_idx1723791.csv
File saved in location: testVisual\multiple\debug_iter2_idx1723791.csv
Saved: output_iter2_idx172379.csv and debug_iter2_idx172379.csv
Processing index: 200619 (iteration 3)
[OK] Total de oraciones: 5


100%|██████████| 5/5 [00:00<00:00,  6.21it/s]


File saved in location: testVisual\multiple\output_iter3_idx2006191.csv
File saved in location: testVisual\multiple\debug_iter3_idx2006191.csv
Saved: output_iter3_idx200619.csv and debug_iter3_idx200619.csv
Processing index: 27573 (iteration 4)
[OK] Total de oraciones: 10


100%|██████████| 10/10 [00:01<00:00,  6.59it/s]

File saved in location: testVisual\multiple\output_iter4_idx275731.csv
File saved in location: testVisual\multiple\debug_iter4_idx275731.csv
Saved: output_iter4_idx27573.csv and debug_iter4_idx27573.csv


#### Prueba de creación de DF único

In [26]:
def extract_features_from_dataset(df_original, sample_size=None):
    """
    Extrae features estilométricos a nivel de oración.
    
    Returns:
        DataFrame con estructura: id_original, model, domain, sentence_num, text, features...
    """
    if sample_size:
        df_original = df_original.sample(n=sample_size, random_state=42)
    
    # Inicializar StyloMetrix (sin guardar archivos)
    stylo = sm.StyloMetrix('en', debug=False)  # debug=False para evitar archivos
    
    all_results = []
    
    for idx, row in df_original.iterrows():
        # Dividir en oraciones (en memoria)
        sentences = split_text_into_sentences(row['generation'])
        
        # Extraer features para todas las oraciones del documento
        features_df = stylo.transform(sentences)
        
        # Agregar metadatos del documento original
        features_df.insert(0, 'id_original', row['id'])
        features_df.insert(1, 'model', row['model'])
        features_df.insert(2, 'domain', row['domain'])
        features_df.insert(3, 'sentence_num', range(len(sentences)))
        # La columna 'text' ya existe en features_df (viene de stylo.transform)
        
        all_results.append(features_df)
    
    # Concatenar todos los resultados
    final_df = pd.concat(all_results, ignore_index=True)
    
    return final_df

In [27]:
features_df = extract_features_from_dataset(generation_sample)

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[OK] Total de oraciones: 23


  0%|          | 0/23 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
  4%|▍         | 1/23 [00:00<00:02,  8.16it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 23/23 [00:01<00:00, 12.19it/s]


[OK] Total de oraciones: 5


  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 5/5 [00:00<00:00, 14.83it/s]


...

  0%|          | 0/19 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
  5%|▌         | 1/19 [00:00<00:01,  9.66it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 19/19 [00:01<00:00,  9.64it/s]


[OK] Total de oraciones: 5


  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 5/5 [00:00<00:00,  6.86it/s]



[OK] Total de oraciones: 10


  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
 10%|█         | 1/10 [00:00<00:01,  6.09it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 10/10 [00:01<00:00,  5.27it/s]


In [ ]:
# Uso:
print(f"Shape: {features_df.shape}")
display(features_df.head(62))

Shape: (62, 201)


id_original    model     domain  sentence_num  \
0   c4059838-c14e-4b84-b75c-12bc0bd2f34a   cohere      books             0   
1   c4059838-c14e-4b84-b75c-12bc0bd2f34a   cohere      books             1   
2   c4059838-c14e-4b84-b75c-12bc0bd2f34a   cohere      books             2   
3   c4059838-c14e-4b84-b75c-12bc0bd2f34a   cohere      books             3   
4   c4059838-c14e-4b84-b75c-12bc0bd2f34a   cohere      books             4   
..                                   ...      ...        ...           ...   
57  4a64da9c-2df8-4f58-91ee-2f9192cc5667  mistral  abstracts             5   
58  4a64da9c-2df8-4f58-91ee-2f9192cc5667  mistral  abstracts             6   
59  4a64da9c-2df8-4f58-91ee-2f9192cc5667  mistral  abstracts             7   
60  4a64da9c-2df8-4f58-91ee-2f9192cc5667  mistral  abstracts             8   
61  4a64da9c-2df8-4f58-91ee-2f9192cc5667  mistral  abstracts             9   

                                                 text  POS_VERB  POS_NOUN  \
0   The story centers on Charles, the husband of A...  0.200000  0.200000   
1   He insists that she stay in the chateau while ...  0.200000  0.133333   
2   Charles abruptly ends his trip and returns hom...  0.300000  0.100000   
3   He announces that he has sold the estate, and ...  0.375000  0.062500   
4   Then he hands Alice a key he says he found on ...  0.235294  0.176471   
..                                                ...       ...       ...   
57  The recent surge of attention to medical image...  0.161290  0.387097   
58  Deep network architectures are particularly go...  0.125000  0.333333   
59  Deep network training is now affordable with t...  0.066667  0.366667   
60  The emergence of self-supervised and unsupervi...  0.176471  0.382353   
61  The promise of this research is to enable the ...  0.193548  0.290323   

     POS_ADJ   POS_ADV   POS_DET  ...   RE  ASF  ASM        OM  RCI  DMC   OR  \
0   0.066667  0.000000  0.133333  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
1   0.000000  0.000000  0.066667  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
2   0.000000  0.200000  0.000000  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
3   0.000000  0.062500  0.062500  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
4   0.000000  0.058824  0.176471  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
..       ...       ...       ...  ...  ...  ...  ...       ...  ...  ...  ...   
57  0.096774  0.000000  0.129032  ...  0.0  0.0  0.0  0.032258  0.0  0.0  0.0   
58  0.208333  0.125000  0.000000  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
59  0.133333  0.033333  0.033333  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
60  0.176471  0.029412  0.058824  ...  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   
61  0.096774  0.000000  0.129032  ...  0.0  0.0  0.0  0.032258  0.0  0.0  0.0   

         QAS   PA   PR  
0   0.200000  0.0  0.0  
1   0.133333  0.0  0.0  
2   0.000000  0.0  0.0  
3   0.062500  0.0  0.0  
4   0.117647  0.0  0.0  
..       ...  ...  ...  
57  0.064516  0.0  0.0  
58  0.000000  0.0  0.0  
59  0.033333  0.0  0.0  
60  0.058824  0.0  0.0  
61  0.096774  0.0  0.0  

[62 rows x 201 columns]